In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [ ]:
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow")


os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "rppawar491"
os.environ["MLFLOW_TRACKING_PASSWARD"] = "024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12e"

#### Entity

In [17]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    x_test_data_path : Path
    y_test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    mlflow_uri : str


#### Configuration Manager

In [18]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.logisticRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            mlflow_uri = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
        )
        
        return model_evaluation_config

In [21]:
import mlflow
import os

# Set tracking URI
mlflow.set_tracking_uri("https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow")

# Set credentials using environment variables
os.environ['MLFLOW_TRACKING_USERNAME'] = 'rppawar491'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12e'

mlflow.set_experiment("LogisticRegression")


2025/02/11 20:58:24 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegression' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/91e2e297a676434099787855510cd3cd', creation_time=1739287708346, experiment_id='0', last_update_time=1739287708346, lifecycle_stage='active', name='LogisticRegression', tags={}>

#### Component

In [22]:
import mlflow
import joblib
import pandas as pd
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, f1_score

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        try:
            accuracy = accuracy_score(actual, pred)
            precision = precision_score(actual, pred, average='weighted')
            f1 = f1_score(actual, pred, average='weighted')
            return accuracy, precision, f1
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            return 0, 0, 0

    def log_into_mlflow(self):
        try:
            test_data = pd.read_csv(self.config.x_test_data_path)
            model = joblib.load(self.config.model_path)

            test_x = test_data
            test_y = pd.read_csv(self.config.y_test_data_path)

            # Set the MLflow tracking URI and pass the access token
            mlflow.set_registry_uri(self.config.mlflow_uri)
            mlflow.set_tracking_uri(self.config.mlflow_uri)

            # Ensure the tracking URI is properly set
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            print(f"Tracking URI: {mlflow.get_tracking_uri()}")

            with mlflow.start_run():
                predicted_classes = model.predict(test_x)

                accuracy, precision, f1 = self.eval_metrics(test_y, predicted_classes)
                scores = {"accuracy": accuracy, "precision": precision, "f1": f1}

                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(scores)

                # Model registry does not work with file store
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegression")
                else:
                    mlflow.sklearn.log_model(model, "model")

                return scores

        except Exception as e:
            raise e


In [25]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

2025-02-11 21:02:44,877: INFO: comman: yaml file : config\config.yaml lodded successfully
2025-02-11 21:02:44,878: INFO: comman: yaml file : params.yaml lodded successfully
2025-02-11 21:02:44,886: INFO: comman: yaml file : schema.yaml lodded successfully
2025-02-11 21:02:44,889: INFO: comman: created directory at :artifacts
2025-02-11 21:02:44,890: INFO: comman: created directory at :artifacts/model_evaluation
Tracking URI: https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow


d:\software\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025/02/11 21:02:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LogisticRegression' already exists. Creating a new version of this model...
2025/02/11 21:02:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression, version 3
Created version '3' of model 'LogisticRegression'.


🏃 View run popular-elk-72 at: https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow/#/experiments/0/runs/6020b592d53349cfa5e98bc4dc4ea8ba
🧪 View experiment at: https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow/#/experiments/0


In [ ]:
#$env:MLFLOW_TRACKING_URI="https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
#$env:MLFLOW_TRACKING_USERNAME="rppawar491"
#$env:MLFLOW_TRACKING_PASSWORD="024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12"